## Yoga Bookings Web Application built on Flask

first one needs to install Flask via prompt typing: 
pip install flask



In [1]:
# Importing the libraries needed

from flask import Flask, render_template, request, session, redirect, url_for
from datetime import timedelta 
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import groupby
from io import BytesIO
import base64

Further we provide screen sample slides with
SQL queries functions described. There will be
six pages/tabs:
- `log in/register`
- `my profile`
- `classes`
- `schedule`
- `instructors`
- `workout history`

### Initiating the app

The Flask object from the flask package initiates the app.

In [2]:
# first the app object is created 

app = Flask(__name__)
app.config['SECRET_KEY'] = 'gatvshbeybbsk'
app.config['PERMANENT_SESSION_LIFETIME'] = timedelta(minutes = 3)

# Next we start creating the subpages of the webapp: 
# The first page shall contain the registration form for new members


## Tab 1. Login & Register
### Functions & SQL queries

First we define the functions necessary for logging in and registering new members. 
Those are: `check_member` and `register_member`.

***CHECK MEMBER***

In [3]:
# Function which checks the credentials 

def check_member(email, password):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('Select member_email, password FROM Members WHERE member_email=? and password=?', (email, password))
        
    result = c.fetchone()
    if result: 
        return True
    else: 
        return False
    
    conn.commit()
    conn.close()

***REGISTER MEMBER***

In [4]:
# Function for registering new members 

def register_member(member_details):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    sql_string = 'INSERT INTO Members (member_name, member_surname, member_email, year_of_birth, membership, password) VALUES (?,?,?,?,?,?)'
    c.execute(sql_string, member_details)
    conn.commit()
    conn.close()

### Page setup

In [5]:
# Define the home page

@app.route('/')
def index():
    return render_template('login.html')


In [6]:
# Registration 
 
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == "POST":
        member_details = (
        request.form['member_name'], 
        request.form['member_surname'],
        request.form['member_email'], 
        request.form['year_of_birth'], 
        request.form['membership'], 
        request.form['password'])
        
        register_member(member_details)
        return redirect(url_for('login'))
    
    else: 
        return render_template('register.html')

In [7]:
# Login  

@app.route('/login', methods = ['GET', 'POST'])
def login():
    if request.method == 'POST':
    
        # Retrieve email and password ceredntials from html form 
        user = request.form['member_email']
        password = request.form['password']
        print(check_member(user, password))
        
        if check_member(user, password):
            session['user'] = user 
            session.permanent = True   
        return redirect(url_for('profile'))
    
    else:
        if 'user' in session: 
            return redirect(url_for('index'))
        
        return render_template('login.html')

In [8]:
# Logout 

@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect(url_for('index'))

## Tab 2. My profile
### Functions & SQL queries


First we need to define a few functions wrapping into them the sql queries needed to operate for each functionality of the page.

In `My profile` tab the user should be able to: 

- check out his/her membership plan
- update membership plan
- view class reservations
- cancel class reservations

Now we want to create a function which will contain the sql query.

***CHECK MEMBERSHIP***

In [9]:
# code here
def check_plan(user): 
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('''
    CREATE VIEW IF NOT EXISTS member_plan
    AS
    SELECT
    id_member, 
    membership, 
    Memberships.membership_name AS name
    FROM 
    Members
    INNER JOIN Memberships ON Members.membership=Memberships.id_membership
    ''')
    c.execute('''
    SELECT name FROM member_plan WHERE (SELECT id_member FROM Members WHERE member_email=?)=id_member
    ''', [user])
    plan = c.fetchone()
    return plan
    conn.commit()
    conn.close()

In [10]:
# plan = check_plan('oliviakul@gmail.com')

***UPDATE PLAN***

In [11]:
def update_plan(user, new_plan):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('UPDATE Members SET membership=? WHERE member_email=?', (new_plan, user))
    conn.commit()
    conn.close()

In [12]:
# sample to check : update_plan('oliviakul@gmail.com', 3)

***VIEW RESERVATIONS***

In [13]:
def view_reservation(): 
    user = str(session['user'])
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()               
    query = '''
    SELECT 
        id_member AS Member,
        id_reservation AS ReservationID, 
        date_hour AS Time, 
        class_name AS Class, 
        instructor_name AS InstructorName, 
        instructor_surname AS InstructorSurname, 
        status AS RegistrationStatus
    FROM Reservations r 
    INNER JOIN Schedule s
    ON r.class_key=s.class_key
    INNER JOIN Classes c
    ON c.id_class=s.id_class
    INNER JOIN Instructors i 
    ON s.id_instructor=i.id_instructor
    WHERE r.id_member=(SELECT id_member FROM Members WHERE member_email = ?)
    '''
    c.execute(query, [user])
    reservation_data = c.fetchall()
    return reservation_data
    conn.commit()
    conn.close()

***CANCEL RESERVATION***

When the reservation is cancelled, the following changes are up: 
- `Reservations.status` changes to `cancelled`
- in `Schedule` fields `available` and `enrolled` are corrected

In [14]:
# as id_resrvation in Reservations is primary key, it will be enough to pass only it as an argument 

def cancel_reserv(id_reservation):
    
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    trigger = '''CREATE TRIGGER IF NOT EXISTS cancel_reservation AFTER UPDATE OF status
    ON Reservations
    WHEN old.status <> new.status
    BEGIN 
    UPDATE Schedule SET enrolled = enrolled - 1 WHERE (SELECT class_key FROM Reservations)= (SELECT class_key FROM Schedule);
    UPDATE Schedule SET available = available + 1 WHERE (SELECT class_key FROM Reservations)= (SELECT class_key FROM Schedule);
    END
    '''
    c.execute(trigger)
    query = '''
    UPDATE Reservations SET status = 'cancelled' where id_reservation=? 
    '''
    c.execute(query, [id_reservation])
    conn.commit()
    conn.close()
   

### Page Setup

In [15]:
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    user = str(session['user'])
    
    # updating plan if requested
    if request.method == 'POST' and 'new_plan' in request.form:
        new_plan = request.form['new_plan']
        update_plan(user, new_plan)
        return redirect('/profile')
    
     # cancel reservation if requested 
    if request.method == 'POST' and 'id_reservation' in request.form:
        id_reservation = request.form['id_reservation']
        cancel_reserv(id_reservation)
        return redirect('/profile')
    
    plan = check_plan(user)
    rows = view_reservation()
    return render_template('profile.html', reservation_data = rows, plan = plan)   

## Tab 3. Classes

In [16]:
@app.route('/classes')
def classes(): 
    return render_template('classes.html')

## Tab 4. Schedule
### Functions & SQL queries

In [17]:
def schedule():
    db_yoga = 'yoga.db'
    conn = sqlite3.connect(db_yoga)
    c = conn.cursor()
    schedule = c.execute("""
    SELECT DISTINCT class_key, class_name, date_hour, instructor_name, place_limit, available
    FROM Instructors i
    INNER JOIN Schedule s
    ON i.id_instructor=s.id_instructor
    INNER JOIN Classes c
    ON s.id_class=c.id_class
    ORDER BY class_key
    """).fetchall()
    conn.commit()
    conn.close()
    return schedule

In [18]:
# Update slots available in Schedule upon booking

def book_class(class_key):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute("""
    UPDATE Schedule SET enrolled = enrolled + 1, available = available - 1 WHERE class_key=?
    """,[class_key])
    c.execute("""
    UPDATE Schedule SET available = 0 WHERE available < 1;
    """)
    conn.commit()
    conn.close()

In [19]:
#def book_class_res(class_key):
#    user = str(session['user'])    
#    c.execute("""
#    INSERT INTO Reservations (id_member, class_key, id_reservation, status) VALUES (id_member = ?, class_key = ?, NULL, 'active');
#    """,[user],[class_key])
#    conn.commit()
#    conn.close()

### Page Setup

In [20]:
@app.route('/schedule', methods = ['GET', 'POST'])
def schedule_table():
    user = str(session['user'])
    
    if request.method == 'POST' and 'class_key' in request.form:
        class_key = request.form['class_key']
        book_class(class_key)
        return redirect('/schedule')    
    
    schedule_info = schedule()
    return render_template('schedule.html', schedule_info = schedule_info)

## Tab 5. Instructors
### Functions & SQL queries

In [21]:
### Instructors page

def instructors():
    db_yoga = 'yoga.db'
    conn = sqlite3.connect(db_yoga)
    c = conn.cursor()
    instructors_info = c.execute("""
    SELECT DISTINCT instructor_name, instructor_surname, instructor_email, class_name 
    FROM Instructors i
    INNER JOIN Schedule s
    ON i.id_instructor=s.id_instructor
    INNER JOIN Classes c
    ON s.id_class=c.id_class
    """).fetchall()
    conn.commit()
    conn.close()
    return instructors_info

#  Create empty dictionary pair and then use a for loop to group the results from 
# instructors_info by the instructor name 
# (groupby function can be imported from itertools library)
# a represents instructors names, which are extracted using lambda f
# t: t['name'], that takes the class name and returns the instructor name
# b represents the group that contains the class name of each instructor name

instructors_info = instructors()
lists = {}

for a, b in groupby(instructors_info, key = lambda t: t[0]):
    lists[a] = list(b)

print(lists.items())

for list_, items in lists.items():
    print(list_)
    for item in items:
        print('     ', item[3])

dict_items([('Cindy', [('Cindy', 'Crawford', 'cindy@gmail.com', 'Yoga for beginners')]), ('Naomi', [('Naomi', 'Campbell', 'blackpanther@gmail.com', 'Kundalini yoga')]), ('Claudia', [('Claudia', 'Schiffer', 'claudiaschiffer@gmail.com', 'Stretching')])])
Cindy
      Yoga for beginners
Naomi
      Kundalini yoga
Claudia
      Stretching


### Page Setup

In [22]:
@app.route('/instructors')
def instructors_table():
    instructors_info = instructors()
    return render_template('instructors.html', instructors_info = instructors_info)

## Tab 5. History

### Page Setup

In [23]:
@app.route('/history')
def fig():
    stat = pd.DataFrame(schedule())
    stat.columns = ['ID','Class','Time_slot', 'I_name','Limit','Avail']
    stat.groupby('Class')['Avail','Limit'].sum().plot.barh(align = 'center', color = ['tab:blue','tab:red'])
    plt.ylabel('Classes')
    plt.xlabel('Avail/Limit summary for this week')
    img = BytesIO()
    plt.savefig(img, format='png')
    plt.close()
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode('utf8')
    return render_template('history.html', plot_url=plot_url)

### Running the app

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2023 20:50:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 20:50:45] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [19/Jan/2023 20:50:45] "GET /profile HTTP/1.1" 200 -


True


127.0.0.1 - - [19/Jan/2023 20:50:54] "GET /classes HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 20:50:59] "GET /instructors HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 20:51:01] "GET /schedule HTTP/1.1" 200 -
C:\Users\1\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """
127.0.0.1 - - [19/Jan/2023 20:51:02] "GET /history HTTP/1.1" 200 -
